
# <p style="text-align: center;">MIS 382: Advanced Predictive Modeling</p>
# <p style="text-align: center;">Assignment 3</p>
## <p style="text-align: center;">Paige McKenzie (pam2932), Yannick Heard (yoh64)</p>
## <p style="text-align: center;">Total points: 60</p>
## <p style="text-align: center;">Due: Wednesday, October 25th, submitted via Canvas by 11:59 pm</p>

Your homework should be written in a **Jupyter notebook**. You may work in groups of two if you wish. Only one student per team needs to submit the assignment on Canvas.  But be sure to include name and UTID for both students.  Homework groups will be created and managed through Canvas, so please do not arbitrarily change your homework group.  

Also, please make sure your code runs and the graphics (and anything else) are displayed in your notebook before submitting. (%matplotlib inline)

# Question 1 - Gradient Descent (5+8+2=15pts)

In this question you will implement vanilla SGD and an adaptive gradient update technique called Adagrad. In addition, you will also implement ridge regularization. 

1. Using stochastic gradient descent, derive the coefficent updates for all 4 coefficients of the model: $$ y = w_0 + w_1*x_1 + w_2*x_1*x_2 + w_3*x_2 $$ Hint: start from the cost function (Assume sum of squared error). If you write the math by hand, submit that as a separate file and make a reference to it in your notebook or include the image in your notebook.
2. Write Python code for an SGD solution to the non-linear model $$ y = w_0 + w_1*x_1 + w_2*x_1*x_2 + w_3*x_2$$ Try to format similarly to scikit-learn's models. Your class should take as input the learning_rate, regularization_constant and number of epochs. The fit method must take as input X,y and a choice of update_rule as 'SGD' or 'adagrad' (Notes on implementation below). The _predict_ method takes an X value (optionally, an array of values). Use your new gradient descent regression to predict the data given in 'samples.csv', for 10 epochs, using learning rates: [.0001, .001, .01, 0.1, 1, 10, 100] and regularization constants in the range: [0,10,100] . Plot MSE and the $w$ parameters as a function of epoch count (10 epochs) for the best 2 combinations of learning_rate and regularization for both SGD and Adagrad. ie you should have 2 plots of MSE and parameter updates for SGD and adagrad each. Report the MSE at the end of 10 epochs for all 4 combinations.
3. Based on the experiments, answer the following:

1. Which of the 2 techniqes allows for larger initial setting of the learning_rate? Why?
2. What would a drawback of adagrad be? How could this be fixed? (Hint: Adadelta)

# Notes on Adagrad
Adagrad (http://ruder.io/optimizing-gradient-descent/) differs from vanilla SGD in that the learning rate of each weight changes over updates. A cache is maintained that holds the sum of squares of all gradients upto the current update. The learning_rate is divided by the cache, resulting in a different learning rate for each weight. A consequence of this update rule is that weights that have already seen large gradients (made large jumps) make smaller updates in subsequent iterations.
Specifically, the steps can be listed as below:
1. cache = cache + (gradients^2)
2. weights = weights + ((learning_rate)/sqrt(cache+1e-6))*gradients

# (Optional) Notes on Training with Gradient Descent
1. Compute error: This consists of a prediction error and a regularization term. From an implementation perspective, this is a function that takes as input the truth, prediction and regularization hyperparameter and returns an error
2. Compute gradients: Take a derivative of the error in terms of the weights. This can be modelled as a function that takes as input the error and features and returns the gradients for each weight
3. Update weights: Weight updates can be done using vanilla SGD or adaptive techniques. The update function takes as inputs the gradient and hyperparameters and returns the new weights.

## Part 1
Cost function:
$$ E(\vec{x}) = \sum_{i=1}^{n} (\vec{w}^{T}\vec{x_{i}}-y_{i})^2+\lambda~||w||^{2}$$
$$ E(\vec{x}) = \sum_{i=1}^{n} (w_0 + w_{1}x_{1i} + w_{2}x_{1i}x_{2i} + w_{3}x_{2i}-y)^2 + \lambda(w_{0}^2+w_{1}^2+w_{2}^2+w_{3}^2)$$

Partial derivatives with respect to weights:
$$ \frac{\partial E(\vec{x})}{\partial w_0} = 2(w_0 + w_{1}x_{1} + w_{2}x_{1}x_{2} + w_{3}x_{2}-y)+2\lambda w_{0}$$
$$ \frac{\partial E(\vec{x})}{\partial w_1} = 2(w_0 + w_{1}x_{1} + w_{2}x_{1}x_{2} + w_{3}x_{2}-y)x_{1}+2\lambda w_{1}$$
$$ \frac{\partial E(\vec{x})}{\partial w_2} = 2(w_0 + w_{1}x_{1} + w_{2}x_{1}x_{2} + w_{3}x_{2}-y)x_{1}x_{2}+2\lambda w_{2}$$
$$ \frac{\partial E(\vec{x})}{\partial w_3} = 2(w_0 + w_{1}x_{1} + w_{2}x_{1}x_{2} + w_{3}x_{2}-y)x_{2}+2\lambda w_{3}$$

Weight update equation:
$$ \Delta w_{i} = -\eta\frac{\partial E(\vec{x})}{\partial w_i} $$

$$ w_{new} = w_{old}+\Delta w_{old} $$

# Part 2

In [1]:
# Initialize variables

import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
%pylab inline
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('samples.csv', index_col=0)

X = normalize(np.array(df[['x_1','x_2']]))
y = np.array(df['y'])

learning_rates = [.0001, .001, .01, 0.1, 1, 10, 100]
lambda_constants = [0, 10, 100]
methods = ['SGD', 'Adagrad']
num_epochs = 10

Populating the interactive namespace from numpy and matplotlib


In [2]:
def make_plots(list, mse):
    #[learning_rate, lambda_constant, method, result, weights_array]
    plt.figure(1, figsize=(16, 8))

    plt.subplot(121)
    plt.plot(np.transpose(list[-1]), marker='o')
    plt.legend(['W0', 'W1', 'W2', 'W3'])
    plt.title('Weights versus Epoch')

    plt.subplot(122)
    plt.plot([i for i in range(1,11)], list[-2], marker='o')
    plt.title('MSE versus Epoch')
    
    plt.suptitle('MSE of {} produced by {} method, with {} learning rate and {} regularization constant'.format(mse, list[2], list[0], list[1]))
    plt.show()

In [3]:
def model(X, y, learning_rate, lambda_const, method, num_epochs):
    weights = [0, 0, 0, 0] #w0, w1, w2, w3 starting point
    if method=='Adagrad':
        cache = [0, 0, 0, 0] #initialize cache to zero for each weight
    epoch_error = [] #store MSE for each epoch
    weights_array = np.transpose(np.array([weights]))
    for epoch in range(num_epochs):
        error_list = [] #store error values to calculate RMSE
        for i in range(len(X)): #iterate through data
            x = X[i]
    
            #predict y value
            error = (predict(weights, x)-y[i])**2
            error_list.append(error)
    
            #update weights
            weights = update_weights_sgd(x, y[i], weights, lambda_const, learning_rate)
            
        #maintain array of weights
        if type(weights_array)==int:
            weights_array = np.transpose(np.array([weights]))
        else:
            weights_array = append(weights_array, np.transpose(np.array([weights])), axis=1)
        #maintain array of error
        epoch_error.append(sum(error_list)/len(error_list))
    return epoch_error, weights_array

In [4]:
def predict(weights, x):
    return weights[0]+weights[1]*x[0]+weights[2]*x[0]*x[1]+weights[3]*x[1]

def gradient(x, y, weights, lambda_const):
    grad_w0 = 2*(weights[0]+weights[1]*x[0]+weights[2]*x[0]*x[1]+weights[3]*x[1]-y)+2*lambda_const*weights[0]
    grad_w1 = 2*(weights[0]+weights[1]*x[0]+weights[2]*x[0]*x[1]+weights[3]*x[1]-y)*x[0]+2*lambda_const*weights[1]
    grad_w2 = 2*(weights[0]+weights[1]*x[0]+weights[2]*x[0]*x[1]+weights[3]*x[1]-y)*x[0]*x[1]+2*lambda_const*weights[2]
    grad_w3 = 2*(weights[0]+weights[1]*x[0]+weights[2]*x[0]*x[1]+weights[3]*x[1]-y)*x[1]+2*lambda_const*weights[3]
    return [grad_w0, grad_w1, grad_w2, grad_w3]

def update_weights_sgd(x, y, weights, lambda_const, learning_rate):
    grad = gradient(x, y, weights, lambda_const)
    return [weights[i]-learning_rate*grad[i] for i in range(len(weights))]

def update_weights_adagrad(x, y, weights, lambda_const, learning_rate, cache):
    grad = gradient(x, y, weights, lambda_const)
    cache = [cache[i]+grad[i]**2 for i in range(len(cache))]
    return cache, [weights[i]-(learning_rate)/((cache[i]+1e-6)**.5)*grad[i] for i in range(len(weights))]

In [8]:
for method in methods:
    mins = []
    stuff = []
    for lambda_constant in lambda_constants:
        learning_rate = .01
        result, weights_array = model(X, y, learning_rate, lambda_constant, method, num_epochs)
        mse = sum(result)/len(result)
        print method, 'learning rate:', learning_rate, 'reg constant:', lambda_constant, 'mse:', mse

SGD learning rate: 0.01 reg constant: 0 mse: 435.299488776
SGD learning rate: 0.01 reg constant: 10 mse: 1462.36715478
SGD learning rate: 0.01 reg constant: 100 mse: 5.33940047542e+189
Adagrad learning rate: 0.01 reg constant: 0 mse: 435.299488776
Adagrad learning rate: 0.01 reg constant: 10 mse: 1462.36715478
Adagrad learning rate: 0.01 reg constant: 100 mse: 5.33940047542e+189


In [ ]:
#actually do the stuff

for method in methods:
    mins = []
    stuff = []
    for lambda_constant in lambda_constants:
        for learning_rate in learning_rates:
            result, weights_array = model(X, y, learning_rate, lambda_constant, method, num_epochs)
            mse = sum(result)/len(result)
            if len(mins)<2:
                mins.append(mse)
                stuff.append([learning_rate, lambda_constant, method, result, weights_array])
            elif min(mins)>mse:
                idx = np.argmin(mins)
                mins[idx] = mse
                stuff[idx] = [learning_rate, lambda_constant, method, result, weights_array]
                
    for i in range(len(stuff)):
        make_plots(stuff[i], round(mins[i],2))

## Part 3

1. Which of the 2 techniqes allows for larger initial setting of the learning_rate? Why?

Adagrad allows a larger initial setting of the learning rate. The cache method of updating the learning rate means that weights that have already seen large gradients, and made large jumps, make smaller updates in subsequent iterations. This means the Adagrad method is less likely to diverge by 'bouncing' out of a minimum due to too large of a learning rate, as the rate will adapt instead of letting the error bounce up the weight surface to infinity, as regular SGD does.
2. What would a drawback of adagrad be? How could this be fixed? (Hint: Adadelta)

Adagrad accumulates squared gradients, which are always postiive. This means the sum keeps growing during training, regardless of how close or far away the surace is from a minimum. Since Adagrad divides the learning rate by the cache, this can cause the learning rate to shrink to zero, preventing the algorithm from moving along the error surface at all. Adadelta restricts the window of accumulated past gradients to some fixed size, thus preventing the learning rate from going to zero.  

# 2. (6+4=10 pts) Tensor Playground
Visit http://playground.tensorflow.org for this problem

A. From the far right, select "Regression" as the problem type, and select the 2nd of the two data sets ( the right one ).  

   i) What sort of test / training loss do you get if you run it for 200 epochs with the following learning rates: .3, .01 and .003 ?  What if you run it for 1000 epochs with these learning rates?  Leave all other values at their defaults ( test/training ratio 50%, Noise 0, Batch Size 10, using Tanh activation function, and No Regularization )
   
   ii) Keeping learning rate at .3, Activation at Tanh, with all others at their defaults, and running for 200 epochs.  
     What sort of test/train loss can you achieve using only 1 neuron in the first hidden layer.  What about for 2,3 or 8 neurons?  Provide screen shots of output layer and comment on how the different output fits look and compare with one another.

   iii)Now keeping learning rate at .03 with all others at their defaults, and running for 200 epochs.  
       Compare how the activation functions affect the ouput ( ReLU, Sigmoid, Tanh, Linear ). Provide screen shots of output results and comment.


B. Neural Nets can fit anything.  Now reset to the initial defaults, and select "Classification" as the problem type, and from the Data section, select the bottom right "Spriral" data set.  With the idea of trying to minimize training/testing error, provide solutions to the problem for the following 2 scenarios.  i) Using just the first 2 inputs ( as per default ) and ii) Using all 7 of the inputs.  You may use as many layers as you want, whatever activation, however man neurons.  Provide screen shots which show your full network, output and parameters. Briefly justify your decisions, and comment on difficulties/tradeoffs, what helps/what doesn't,etc. 

## ANSWER

## Part A
### Part i
200 epochs with .3 learning rate produces a test loss of 0.021, and training loss of 0.018.

200 epochs with .01 learning rate produces a test loss of 0.024, and training loss of 0.021.

200 epochs with .003 learning rate produces a test loss of 0.038, and training loss of 0.033.


1000 epochs with .3 learning rate produces a test loss of 0.015, and training loss of 0.013.

1000 epochs with .01 learning rate produces a test loss of 0.019, and training loss of 0.016.

1000 epochs with .003 learning rate produces a test loss of 0.022, and training loss of 0.019.

### Part ii
#### Single Neuron
200 epochs with .3 learning rate and a single neuron in the 1st hidden layer produces a test loss of 0.045 and training loss of 0.038.

In [ ]:
from IPython.display import Image
Image("1 neuron.JPG")

#### Two Neurons
200 epochs with .3 learning rate and two neurons in the 1st hidden layer produces a test loss of 0.031 and training loss of 0.028.

In [ ]:
Image("2 neurons.JPG")

#### Three Neurons
200 epochs with .3 learning rate and three neurons in the 1st hidden layer produces a test loss of 0.019 and training loss of 0.017.

In [ ]:
Image("3 neurons.JPG")

#### Eight Neurons
200 epochs with .3 learning rate and eight neurons in the 1st hidden layer produces a test loss of 0.020 and training loss of 0.015.

In [ ]:
Image("8 neurons.JPG")

#### Commentary
Visually, a single neuron produces a linear split of the data while two neurons produces a mild striped pattern. Both 3 and 8 neurons produce a similar pattern, as they are sufficient to appropriately segregate the data.

### Part ii
#### Relu

In [ ]:
Image("ReLU.JPG")

#### Sigmoid

In [ ]:
Image("sigmoid.JPG")

#### Tanh

In [ ]:
Image("tanh.JPG")

#### Linear

In [ ]:
Image("linear.JPG")

## Part B

### i)
For using on the first two inputs the best combination found was 4 layers each with 6 neurons in them. 
The difficulties or tradeoff of this model was that it was definitely slower than those models with lower numbers of layers and neurons. While only using two input features models that had lower numbers of layers and neurons appeared to struggle more to be able to fit the complexity of the data. Models that involved more layers and neurons appeared to have the ability to fit the data more effectively, particularly in regards to the number of neurons in each layer. We decided to use this higher level of neurons and and layers in order to be able try and fully accomodate for the complex form of the data, with a limited number of features. 

### ii)
For using all of the seven available inputs the best combination found was 2 layers with 8 neurons and then 2 neurons. 
The tradeoffs or difficulties of using this model are that more complex models do have the capability to fit the model more accurately, however additional layers and neurons rapidly increase the amount of time that each iteration. Since models with higher numbers of layers increased the iteration time, and only increased the accuracy by a tiny amount we decided to utilize a model with less layers and neurons.

In [ ]:
Image("2nodes.png")

In [ ]:
Image("7nodes.png")

# Question 3: Multi-layer Perceptron Regressor (15 points)

In this question, you will explore the application of Multi-layer Perceptron (MLP) regression using sklearn package in Python. We will use the Appliances energy prediction dataset for this problem https://archive.ics.uci.edu/ml/datasets/Appliances+energy+prediction.

Following code will pre-process the data and split the data into training and test set using [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) with **random state 42** and **test_size = 0.33**.
Make sure you have 56 variables and one target variable.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,KFold)
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import glob
%matplotlib inline

from sklearn import datasets

data = pd.read_csv('./energydata_complete.csv') 

y = data['Appliances']
X = data.drop(['date','Appliances', 'rv1', 'rv2'], axis = 1)


from dateutil import parser
timeData = np.array(data['date'])

days = []
hours = []
for line in xrange(len(timeData)):
    day = parser.parse(timeData[line]).weekday()
    hour = parser.parse(timeData[line]).hour
    days.append(day)
    hours.append(hour)
    
X = pd.concat([X, pd.get_dummies(days), pd.get_dummies(hours)], axis = 1)


X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33, random_state=42)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

One more thing to use in this problem is [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). Instead of fitting a model on original data, use StandardScaler to make each feature centered ([Example](http://scikit-learn.org/stable/auto_examples/applications/plot_prediction_latency.html#sphx-glr-auto-examples-applications-plot-prediction-latency-py)). Whenever you have training and test data, fit a scaler on training data and use this scaler on test data. Here, scale only features (independent variables), not target variable y.

Use [sklearn.neural_nework.MLPRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor) to do a 5-fold cross validation using sklearn's [KFold](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold). The cross validation must be performed on the **training data**. 

1) Use following parameter settings for MLPRegressor:

    activation = 'tanh', solver = 'sgd', learning_rate='constant', random_state=42,
    batch_size=5000, learning_rate_init = 0.005
    
Now, consider three different settings for the number of hidden units:
    
   (a) *hidden_layer_sizes = (2,)* (b) *hidden_layer_sizes = (30,10)* (c) *hidden_layer_sizes = (100,)*
    
   Report the average Root Mean Squared Error (RMSE) value based on your 5-fold cross validation for each model: (a), (b), and (c) (5pts)
      
2) Now, using the same number of hidden units used in part 1), train MLPRegressor models on whole training data and report RMSE score for both Train and Test set (Again, use StandardScaler). Which model works the best, (a), (b), or (c)? Briefly analyze the result in terms of the number of hidden units. (3pts)

3) MLPRegressor has a built-in attribute *loss\_curve\_* which returns the loss at each epoch (misleadingly referred to as "iteration" in scikit documentation, though they use epoch in the actual code!). For example, if your model is named as *my_model* you can call it as *my\_model.loss\_curve\_* ([example](http://scikit-learn.org/stable/auto_examples/neural_networks/plot_mlp_training_curves.html#sphx-glr-auto-examples-neural-networks-plot-mlp-training-curves-py)). Plot three curves for model (a), (b), and (c) in one figure, where *X-axis* is epoch  number and *Y-axis* is squared root of *loss\_curve\_* value. (2pts)

4) Use following parameter settings for MLPRegressor:

    activation = 'tanh', solver = 'sgd', learning_rate='constant', random_state=42,
    batch_size=5000, hidden_layer_sizes = (15,)
    
Now, consider three different settings for the learning rates:
    
   (i) *learning_rate_init = 0.005* (ii) *learning_rate_init = 0.01* (iii) *learning_rate_init = 1*
    
   Report the average Root Mean Squared Error (RMSE) value based on your 5-fold cross validation for each model: (i), (ii), and (iii) (5pts)




## Part 1

In [ ]:
from sklearn.model_selection import KFold

hidden_layer_sizes = [(2,),(30,10),(100,)]

kf = KFold(n_splits=5, shuffle=False, random_state=0)

for i in range(len(hidden_layer_sizes)):
    #iterate through hidden layer settings
    
    fold_error = []
    for train_index, test_index in kf.split(X_train):
        X_kfold_train, y_kfold_train = X_train.iloc[train_index,], y_train.iloc[train_index,]
        X_kfold_test, y_kfold_test = X_train.iloc[test_index,], y_train.iloc[test_index,]
    
        #scale train and test X data
        scaler = StandardScaler()
        scaler.fit(X_kfold_train)
        scaled_X_train=scaler.transform(X_kfold_train)
        scaled_X_test=scaler.transform(X_kfold_test)

        #define the MLP
        MLP = MLPRegressor(hidden_layer_sizes=(hidden_layer_sizes[i]), activation = 'tanh', solver = 'sgd', learning_rate='constant', random_state=42, batch_size=5000, learning_rate_init = 0.005)
        #fit to training data
        MLP.fit(scaled_X_train,y_kfold_train)
        #predict on test data
        pred = MLP.predict(scaled_X_test)
    
        fold_error.append(mean_squared_error(y_kfold_test, pred)**0.5)
    
    #take the average RMSE across fold errors
    error = np.mean(fold_error)
    print 'Hidden layer size of shape {} produces average RMSE of {} based on 5-fold cross-validation\n'.format(hidden_layer_sizes[i], error)

Part C, shape of (100,) produces the lowest RMSE.
## Part 2

In [ ]:
hidden_layer_sizes = [(2,),(30,10),(100,)]

kf = KFold(n_splits=5, shuffle=False, random_state=0)

for i in range(0, len(hidden_layer_sizes)):
    #define a standard scaler
    scaler = StandardScaler()
    scaler.fit(X_train)

    #scale the training and test X data
    scaled_X_train = scaler.transform(X_train)
    scaled_X_test = scaler.transform(X_test)

    #define the MLP
    MLP=MLPRegressor(hidden_layer_sizes=(hidden_layer_sizes[i]), activation = 'tanh', solver = 'sgd', learning_rate='constant', random_state=42, batch_size=5000, learning_rate_init = 0.005) 
    MLP.fit(scaled_X_train,y_train)
    
    #make predictions on test and train data
    pred = MLP.predict(scaled_X_test)
    
    error = mean_squared_error(y_test, pred)**0.5
    
    print 'Hidden layer size of shape {} produces RMSE of {} on test data'.format(hidden_layer_sizes[i], error)

    pred = MLP.predict(scaled_X_train)
    error = mean_squared_error(y_train, pred)**0.5
    print '- the RMSE on training data is {}\n'.format(error)

The first shape has only 1 hidden layer with 2 units, and produces the highest RMSE, probably indicating the model is insufficiencly complex.

The second shape has a hidden layer with 30 units and a hidden layer with 10 units, and produces a moderate RMSE on test data but a dramatic improvement over the first model in training data. This model may be overfitting.

The last shape has a single hidden layer, with 100 units, and produces the best RMSE on test and training data. Clearly, the complexity pays off in this case.

## Part 3

In [ ]:
import math
import matplotlib.pyplot as plt

#prep the plot
fig, ax = plt.subplots()
    
kf = KFold(n_splits=5, shuffle=False, random_state=0)

for i in range(0,len(hidden_layer_sizes)):
    #define a standard scaler
    scaler = StandardScaler()
    scaler.fit(X_train)

    #scale the training and test X data
    scaled_X_train = scaler.transform(X_train)
    scaled_X_test = scaler.transform(X_test)

    #define the MLP
    MLP=MLPRegressor(hidden_layer_sizes=(hidden_layer_sizes[i]), activation = 'tanh', solver = 'sgd', learning_rate='constant', random_state=42, batch_size=5000, learning_rate_init = 0.005) 
    MLP.fit(scaled_X_train,y_train)
    
    loss_values = MLP.loss_curve_
    sqrt_loss=[]
    for j in range(0,len(loss_values)):
        #create list of square roots of loss
        sqrt_loss.append(loss_values[j]**0.5)
    
    #plot stuff
    ax.scatter([k for k in range(1, len(sqrt_loss)+1)], sqrt_loss, label=hidden_layer_sizes[i])

ax.legend()
plt.ylabel('RMSE')
plt.xlabel('Epoch')
plt.title('RMSE vs. Epoch')

plt.show()


## Part 4

In [ ]:
from sklearn.model_selection import KFold

learning_rates = [0.005, 0.01, 1]

#use a seed with K folds for repeatable results
kf = KFold(n_splits=5, shuffle=False, random_state=0)

for i in range(0,len(learning_rates)):
    #iterate through hidden layer settings
    
    fold_error = []
    for train_index, test_index in kf.split(X_train):
        X_kfold_train, y_kfold_train = X_train.iloc[train_index,], y_train.iloc[train_index,]
        X_kfold_test, y_kfold_test = X_train.iloc[test_index,], y_train.iloc[test_index,]
    
        #scale train and test X data
        scaler = StandardScaler()
        scaler.fit(X_kfold_train)
        scaled_X_train = scaler.transform(X_kfold_train)
        scaled_X_test = scaler.transform(X_kfold_test)

        #define the MLP
        MLP=MLPRegressor(activation = 'tanh', solver = 'sgd', learning_rate='constant', random_state=42,
        batch_size=5000, hidden_layer_sizes = (15,), learning_rate_init=learning_rates[i])
    
        MLP.fit(scaled_X_train,y_kfold_train)
    
        pred = MLP.predict(scaled_X_test)
    
        fold_error.append(mean_squared_error(y_kfold_test, pred)**0.5)
    
    #take the average RMSE from a set of cross folds
    error = np.mean(fold_error)
    print 'Initial learning rate of {} produced average RMSE of {} based on 5-fold cross-validation\n'.format(learning_rates[i], error)

# Question 4: (2+5+3=10 pts) Bayes Decision Theory
a. Explain what you understand by class-conditional likelihood, class priors, and posterior probability of a class given an input, and the relationship between them. Please define all symbols and equations used explicitly.
( 2 points )

b. Suppose you want to learn a binary classifier to predict whether or not a customer will buy a TV. The class label is 0 if the customer does not buy a TV and 1 if they do. For each customer, you are given two features, $x_1$ is the per hour salary and $x_2$ is the age. Assume that the class conditional distribution $p([x_1 , x_2]|C)$ is Gaussian. The mean salary and age of the people who do buy a TV is 15 and 30 respectively and that of those who don’t is 8.5 and 25. Assume that both classes of customers have the same covariance given by the Identity matrix. Further, your sales data suggests that only 1 in 3 people actually bought a TV in the last few years. Mathematically derive the (optimal) Bayes decision boundary for this problem. (5 points)

c. Now write code to sample 100 customers from each class (C = 0, 1) under the assumed distribution and the estimated parameters and plot their features. Additionally, plot the decision boundary you obtained in the part (b) on the same plot. (3 points)

## ANSWER

## Part A
Class conditional likelihood represents how likely an x,y pair is given the pair's class, $P(x,y|C_{0})$ or $P(x,y|C_{1})$

Class priors represent how likely we think the class is before looking at data, $P(C_{0})$ or $P(C_{1})$ 

Posterior probability represents how likely a class is, given a certain observation, $P(C_{0}|x,y)$ or $P(C_{1}|x,y)$

From Bayes Rule, we can say that 
$$P(C_{0}|x,y) = \frac{P(C_{0})P(x,y|C_{0})}{P(x,y)}$$

$$P(C_{1}|x,y) = \frac{P(C_{1})P(x,y|C_{1})}{P(x,y)}$$

## Part B
Define $C_{0}$ as the customer not buying a TV, $C_{1}$ as the customer buying a TV, and let x be the salary of a customer, y their age.


The probability of each salary, age combination given a class is given by the normal distributions:
$$P(x,y|C_{0}) = \frac{1}{2\pi}e^{\frac{-1}{2}[(x-8.5)^2+(y-25)^2]}$$
$$P(x,y|C_{1}) = \frac{1}{2\pi}e^{\frac{-1}{2}[(x-15)^2+(y-30)^2]}$$



We also know that $P(C_{0})=\frac{2}{3}$ and $P(C_{1})=\frac{1}{3}$.

The Bayes Discriminant is $g_{i}(x,y) = P(C_{i}|x,y)$ and we are concerned with the decision boundary, where 

$$P(C_{0}|x,y) = P(C_{1}|x,y)$$

$$\frac{P(C_{0})P(x,y|C_{0})}{P(x,y)} = \frac{P(C_{1})P(x,y|C_{1})}{P(x,y)}$$

$$P(C_{0})P(x,y|C_{0}) = P(C_{1})P(x,y|C_{1})$$

$$\frac{2}{3}P(x,y|C_{0}) = \frac{1}{3}P(x,y|C_{1})$$

$$2 \frac{1}{2\pi}e^{\frac{-1}{2}[(x-8.5)^2+(y-25)^2]} = \frac{1}{2\pi}e^{\frac{-1}{2}[(x-15)^2+(y-30)^2]}$$

$$ln(2e^{\frac{-1}{2}[(x-8.5)^2+(y-25)^2]}) = ln(e^{\frac{-1}{2}[(x-15)^2+(y-30)^2]})$$

$$ln2-\frac{1}{2}[(x-8.5)^2+(y-25)^2] = -\frac{1}{2}[(x-15)^2+(y-30)^2])$$

$$ (x-8.5)^2+(y-25)^2 = (x-15)^2+(y-30)^2+ln4$$

$$y = -\frac{13}{10}x+\frac{427.75+ln4}{10}$$

## Part C

In [ ]:
from numpy.random import multivariate_normal
from math import log

def func(x):
    return (-13*x+427.75+log(4))/10.0

mean_1 = np.array([15, 30])
mean_0 = np.array([8.5, 25])
covariance = np.array([[1, 0], [0, 1]])

x,y = np.random.multivariate_normal(mean_0, covariance, 100).T
plt.plot(x,y, 'o', color='blue')
x,y = np.random.multivariate_normal(mean_1, covariance, 100).T
plt.plot(x,y, 'o', color='red')

plt.plot([6, 18], [func(6), func(18)], color='purple')

plt.title('Random Draws, divided by decision boundary')
plt.legend(['Buy', 'Not buy', 'Boundary'])
plt.xlabel('Salary')
plt.ylabel('Age')

plt.show()

# Question 5: Decision Tree using Python (10 pts)
In this problem, you will model the data using decision trees to perform a classification task. Load the BreastTissue.csv dataset. The dataset has been preprocessed. The description of the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/Breast+Tissue). Using the class tree.DecisionTreeClassifier (http://scikit-learn.org/stable/modules/classes.html#module-sklearn.tree), build two different trees with a maximum depth of two using the split criteria (i) Gini and (ii) Entropy. Use all the data to build trees. Plot the two trees. If your classifier object is called clf, use the following commands to save the generated tree as a '.dot' file that can be used to visualize the tree using Webgraphviz: http://www.webgraphviz.com/

Hint: see  http://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html

Copy paste the contents of your '.dot' file into the text window on the website to visualize the trees. At which node(s) do they differ?

In [ ]:
from sklearn import tree
import pandas as pd
from pandas import DataFrame
from sklearn.externals.six import StringIO
from sklearn.tree import DecisionTreeClassifier

data = pd.read_csv('./BreastTissue.csv')
input_data = data.as_matrix(columns = [["I0", 'PA500', "HFS", "DA", 'Area', 'A/DA', 'Max IP', 'DR', 'P']])
target = data.as_matrix(columns = ["Class"])

clf = DecisionTreeClassifier(random_state=0, max_depth = 2)
clf = clf.fit(input_data, target)
tree.export_graphviz(clf, out_file='tree_dot_gini')

clf = DecisionTreeClassifier(criterion = 'entropy',random_state=0, max_depth = 2)
clf = clf.fit(input_data, target)
tree.export_graphviz(clf, out_file = 'tree_dot_entropy')

The two images of trees below are of the two trees created with the different criteria, Gini and Entropy. These two trees differ in the second node as in the Gini model it decides that no further splits are required, while in the Entropy model it splits again to another layer.

In [ ]:
Image("gini.png")

In [ ]:
Image("entropy.png")